# 여러 파일 같은 폴더

In [1]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertConfig
from torch.utils.data import DataLoader, TensorDataset
import openpyxl
from modeling.model import ABSAModel
import os
from glob import glob

# 1. 모델 파일 경로 설정
model_dir = r"/Users/kyn03/Downloads/NIA_ABSA/AI모델/소스코드/AI모델소스코드/ckpt/result_model/"

# 2. 모델 설정 로드
config = BertConfig.from_pretrained(model_dir + "config.json")
config.init_model_path = 'klue/bert-base'
config.sentiment_in_feature = 768
config.aspect_in_feature = 768
config.sentiment_drop_ratio = 0.1
config.aspect_drop_ratio = 0.1

# 3. ABSAModel 클래스 초기화
model = ABSAModel(config, num_sentiment=8, num_aspect=226, num_aspect2=16)

# 4. 모델 가중치 로드
model_weights = model_dir + "pytorch_model.bin"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load(model_weights, map_location=device), strict=False)
model.to(device)

# 5. Tokenizer 로드
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

# 분석할 input 파일들이 있는 폴더 경로
input_folder = r"\Users\kyn03\Downloads\11_12"
output_folder = r"\Users\kyn03\Downloads\11_12\분석"
os.makedirs(output_folder, exist_ok=True)

# 12. 예측 결과 매핑 및 엑셀 기록
def extract_between_zeros(input_list):
    try:
        first_zero_idx = input_list.index(0)
        second_zero_idx = input_list.index(0, first_zero_idx + 1)
        return input_list[first_zero_idx+1:second_zero_idx]
    except ValueError:
        return []

# Aspect Category 매핑 Dictionary 생성
label_map_dict = {
    "가격": ["가격"],
    "품질/디자인/구성": ["품질", "디자인", "색상", "제품구성", "용기", "마감", "포장", "굽", "수납", "품질/디자인/구성",
                  "소재", "제형", "성분", "성분/재질", "재질/소재", "재질/질감"],
    "사이즈/무게/개수": ["사이즈", "무게", "용량", "용량/사이즈", "두께", "치수/사이즈", "길이", "사이즈/부피", "사이즈/용량",
                  "사이즈/폭/길이/두께", "용량/수량", "용량/개수", "사이즈/두께", "길이/폭/두께", "깊이/높이", "수량/개수"],
    "효과/성능/기능": ["기능","조작성","기능/효과","음량/음질","화질","효과/성능/기능","세척/세정력","시간/속도","보습력/수분감",
                 "흡수력","내구성","지속력/유지력","발림성","배터리","세정력/청결감","윤기/피부(톤)","내구성/견고성","지속력",
                 "머릿결관리","염색력","발색력","보습력/수분감/쿨링감","절삭력","신축성","소비전력","탈취/제습력","커버력",
                 "밀착력/접착력","농축도/수용성","탈모개선","스타일링효과","기능성","세팅력/고정력","청량감/쿨링감",
                 "세정력","탄력","코팅력","거품력","씨제거력","수분감/보습력","밀착감/접착력","분사력","클렌징/제거력",
                 "열전도성","흡착/접착력","피부(손)보호","두피보호","살균/소독","제연/냄새방지","방수성","위생/살균/성분","흡·접착력"],
    "사용감/착용감": ["향/냄새", "소음", "향", "착용감", "자극성", "착화감", "핏", "요리력", "촉감/감촉", "촉감", "사용감", "조립성",
                "호흡성/통기성", "냄새", "이염", "그립감", "피부타입"],
    "편의성/활용성": ["편의성", "편의성/사용성/활용도", "편의성/활용성", "활용성", "수납/건조공간", "사용성", "사용성/편의성",
                "연마용이성", "세척용이성", "정리성/수납력"],
    "제조/유통/서비스":["제조일/제조사", "서비스", "유통기한"] 
}

# 대분류 Asepct Category Dictionary에 BIO tag 적용
label_list, label_changing_rule = [], {}
for key in label_map_dict.keys():
    if key != 'O':
        label_list.extend(['B-' + key, 'I-' + key])
    else:
        label_list.append('O')
for key, labels in label_map_dict.items():
    for label in labels:
        for tag in ["B-", "I-"]:
            label_changing_rule[tag + label] = tag + key

# 원래의 sentiment 및 aspect labels 정의
original_sentiment_labels = ["PAD", "O", "B-긍정", "I-긍정", "B-부정", "I-부정", "B-중립", "I-중립"]
original_aspect2_list = ['PAD', 'O', 'B-가격', 'I-가격', 'B-품질/디자인/구성', 'I-품질/디자인/구성', 'B-사이즈/무게/개수', 'I-사이즈/무게/개수', 
                         'B-효과/성능/기능', 'I-효과/성능/기능', 
                         'B-사용감/착용감', 'I-사용감/착용감', 'B-편의성/활용성', 'I-편의성/활용성', 'B-제조/유통/서비스', 'I-제조/유통/서비스']

# 파일별로 반복 처리
for input_file in glob(os.path.join(input_folder, "*.xlsx")):
    original_file_name = os.path.splitext(os.path.basename(input_file))[0]
    df = pd.read_excel(input_file)
    
    # 리뷰 텍스트 추출 및 토크나이징
    text_data = df['리뷰'].dropna().astype(str).tolist()
    inputs = tokenizer(text_data, padding=True, truncation=True, return_tensors="pt")
    ids, mask = inputs['input_ids'], inputs['attention_mask']

    # TensorDataset 및 DataLoader 설정
    dataset = TensorDataset(ids, mask)
    dataloader = DataLoader(dataset, batch_size=4)

    # 모델 예측 수행
    all_sentiments, all_aspects, all_aspects2 = [], [], []
    model.eval()
    with torch.no_grad():
        for batch in dataloader:
            batch = tuple(t.to(device) for t in batch)
            sentiment, aspect, aspect2 = model(ids=batch[0], mask=batch[1])
            all_sentiments.extend(sentiment)
            all_aspects.extend(aspect)
            all_aspects2.extend(aspect2)

    # 엑셀 파일 생성 및 저장
    wb = openpyxl.Workbook()
    ws = wb.active

    # 각 예측 결과를 엑셀 파일에 기록
    for i, (sent, asp, asp2) in enumerate(zip(all_sentiments, all_aspects, all_aspects2)):
        sent = extract_between_zeros(sent)
        asp = extract_between_zeros(asp)
        asp2 = extract_between_zeros(asp2)
        tokenized_text = tokenizer.convert_ids_to_tokens(inputs['input_ids'][i].tolist(), skip_special_tokens=True)
        cleaned_tokenized_text = [tok for idx, tok in enumerate(tokenized_text) if sent[idx] != 0]

        # 엑셀에 결과 기록
        ws.append(["Sentiment Origin Labels"] + sent)
        ws.append(["Sentiment Mapped Labels"] + [original_sentiment_labels[idx] for idx in sent if idx != 0])
        ws.append(["Aspect2 Original Labels"] + asp2)
        ws.append(["Aspect2 Mapped Labels"] + [original_aspect2_list[idx] for idx in asp2 if idx != 0])
        ws.append(["Tokenized Text"] + cleaned_tokenized_text)
        ws.append([])  # 빈 줄로 구분

    # 파일 저장
    output_file_excel = os.path.join(output_folder, f"{original_file_name}.xlsx")
    wb.save(output_file_excel)
    print(f"[INFO] 엑셀 파일 '{output_file_excel}'로 저장 완료.")


C:\Users\kyn03\AppData\Local\Temp\ipykernel_29460\3549630789.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_weights, map_locatio

[INFO] 엑셀 파일 '\Users\kyn03\Downloads\11_12\분석\다시.xlsx'로 저장 완료.


# 달바

In [1]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertConfig
from torch.utils.data import DataLoader, TensorDataset
import openpyxl
# ABSAModel 가져오기
from modeling.model import ABSAModel
import os

# 1. 모델 파일 경로 설정
model_dir = r"/Users/kyn03/Downloads/NIA_ABSA/AI모델/소스코드/AI모델소스코드/ckpt/result_model/"

# 2. 모델 설정 로드
config = BertConfig.from_pretrained(model_dir + "config.json")
config.init_model_path = 'klue/bert-base'
config.sentiment_in_feature = 768
config.aspect_in_feature = 768
config.sentiment_drop_ratio = 0.1
config.aspect_drop_ratio = 0.1

# 3. ABSAModel 클래스 초기화
model = ABSAModel(config, num_sentiment=8, num_aspect=226, num_aspect2=16)

# 4. 모델 가중치 로드
model_weights = model_dir + "pytorch_model.bin"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load(model_weights, map_location=device), strict=False)
model.to(device)

# 5. Tokenizer 로드
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

# 6. 데이터 로드
input_file = r"\Users\kyn03\Downloads\고유id의첫번째리뷰필터링_241030.xlsx"  # 업로드된 파일 경로 사용
original_file_name = os.path.splitext(os.path.basename(input_file))[0]

df = pd.read_excel(input_file)
text_data = df['리뷰'].tolist()

# Check for any non-string values
text_data = df['리뷰'].dropna().astype(str).tolist()
text_data = [str(item) for item in df['리뷰'].dropna().tolist()]


# 7. 텍스트 데이터 토크나이징
inputs = tokenizer(text_data, padding=True, truncation=True, return_tensors="pt")
ids, mask = inputs['input_ids'], inputs['attention_mask']

# 8. TensorDataset 및 DataLoader 설정
dataset = TensorDataset(ids, mask)
dataloader = DataLoader(dataset, batch_size=4)

# 9. Aspect Category 매핑 Dictionary 생성
label_map_dict = {
    "가격": ["가격"],
    "품질/디자인/구성": ["품질", "디자인", "색상", "제품구성", "용기", "마감", "포장", "굽", "수납", "품질/디자인/구성",
                  "소재", "제형", "성분", "성분/재질", "재질/소재", "재질/질감"],
    "사이즈/무게/개수": ["사이즈", "무게", "용량", "용량/사이즈", "두께", "치수/사이즈", "길이", "사이즈/부피", "사이즈/용량",
                  "사이즈/폭/길이/두께", "용량/수량", "용량/개수", "사이즈/두께", "길이/폭/두께", "깊이/높이", "수량/개수"],
    "효과/성능/기능": ["기능","조작성","기능/효과","음량/음질","화질","효과/성능/기능","세척/세정력","시간/속도","보습력/수분감",
                 "흡수력","내구성","지속력/유지력","발림성","배터리","세정력/청결감","윤기/피부(톤)","내구성/견고성","지속력",
                 "머릿결관리","염색력","발색력","보습력/수분감/쿨링감","절삭력","신축성","소비전력","탈취/제습력","커버력",
                 "밀착력/접착력","농축도/수용성","탈모개선","스타일링효과","기능성","세팅력/고정력","청량감/쿨링감",
                 "세정력","탄력","코팅력","거품력","씨제거력","수분감/보습력","밀착감/접착력","분사력","클렌징/제거력",
                 "열전도성","흡착/접착력","피부(손)보호","두피보호","살균/소독","제연/냄새방지","방수성","위생/살균/성분","흡·접착력"],
    "사용감/착용감": ["향/냄새", "소음", "향", "착용감", "자극성", "착화감", "핏", "요리력", "촉감/감촉", "촉감", "사용감", "조립성",
                "호흡성/통기성", "냄새", "이염", "그립감", "피부타입"],
    "편의성/활용성": ["편의성", "편의성/사용성/활용도", "편의성/활용성", "활용성", "수납/건조공간", "사용성", "사용성/편의성",
                "연마용이성", "세척용이성", "정리성/수납력"],
    "제조/유통/서비스":["제조일/제조사", "서비스", "유통기한"]}

# 대분류 Asepct Category Dictionary에 BIO tag 적용
label_list,label_changing_rule = [], {}
for key in label_map_dict.keys():
    if key != 'O':
        label_list.extend(['B-' + key, 'I-' + key])
    else:
        label_list.append('O')
for key, labels in label_map_dict.items():
    for label in labels:
        #if key != label:
            for tag in ["B-", "I-"]:
                label_changing_rule[tag + label] = tag + key

a = ["PAD","O"]
aspect1_labels = list(label_changing_rule.keys())
a.extend(aspect1_labels)
print(a)

# 10. 모델 예측 수행
model.eval()
all_sentiments, all_aspects, all_aspects2 = [], [], []

with torch.no_grad():
    for batch in dataloader:
        batch = tuple(t.to(device) for t in batch)
        sentiment, aspect, aspect2 = model(ids=batch[0], mask=batch[1])
        all_sentiments.extend(sentiment)
        all_aspects.extend(aspect)
        all_aspects2.extend(aspect2)

# 11. 엑셀 파일 생성
wb = openpyxl.Workbook()
ws = wb.active

original_sentiment_labels = ["PAD","O","B-긍정", "I-긍정", "B-부정", "I-부정", "B-중립", "I-중립"]
original_aspect2_list = ['PAD','O','B-가격', 'I-가격', 'B-품질/디자인/구성', 'I-품질/디자인/구성', 'B-사이즈/무게/개수', 'I-사이즈/무게/개수', 
                         'B-효과/성능/기능', 'I-효과/성능/기능', 
                         'B-사용감/착용감', 'I-사용감/착용감', 'B-편의성/활용성', 'I-편의성/활용성', 'B-제조/유통/서비스', 'I-제조/유통/서비스']
# original_aspect1_list = ['PAD', 'O', 
#                          'B-가격', 'I-가격',
#                          'B-품질', 'I-품질', 'B-디자인', 'I-디자인', 'B-색상', 'I-색상', 'B-제품구성', 'I-제품구성', 'B-용기', 'I-용기', 
#                          'B-마감', 'I-마감', 'B-포장', 'I-포장', 'B-굽', 'I-굽', 'B-수납', 'I-수납', 'B-소재', 'I-소재', 'B-제형', 'I-제형', 'B-성분', 'I-성분', 
#                          'B-성분/재질', 'I-성분/재질', 'B-재질/소재', 'I-재질/소재', 'B-재질/질감', 'I-재질/질감', 'B-사이즈', 'I-사이즈', 'B-무게', 'I-무게', 
#                          'B-용량', 'I-용량', 'B-용 량/사이즈', 'I-용량/사이즈', 'B-두께', 'I-두께', 'B-치수/사이즈', 'I-치수/사이즈', 'B-길이', 'I-길이', 
#                          'B-사이즈/부피', 'I-사이즈/부피', 'B-사이즈/용량', 'I-사이즈/용량', 'B-사이즈/폭/길이/두께', 'I-사이즈/폭/길이/두께', 
#                          'B-용량/수량', 'I-용량/수량', 'B-용량/개수', 'I-용량/개수', 'B-사이즈/두께', 'I-사이즈/두께', 'B-길이/폭/두께', 'I-길이/폭/두께', 
#                          'B-깊이/높이', 'I-깊이/높이', 'B-수량/개수', 'I-수량/개수', 'B-기능', 'I-기능', 'B-조작성', 'I-조작성', 'B-기능/효과', 'I-기능/효과', 
#                          'B-음량/음질', 'I-음량/음질', 'B-화질', 'I-화질', 'B-세척/세정력', 'I-세척/세정력', 'B-시간/속도', 'I-시간/속도', 
#                          'B-보습력/수분감', 'I-보습력/수분감', 'B-흡수력', 'I-흡수력', 'B-내구성', 'I-내구성', 'B-지속력/유지력', 'I-지속력/유지력', 
#                          'B-발림성', 'I-발림성', 'B-배터리', 'I-배터리', 'B-세정력/청결감', 'I-세정력/청결감', 'B-윤기/피 부(톤)', 'I-윤기/피부(톤)', 
#                          'B-내구성/견고성', 'I-내구성/견고성', 'B-지속력', 'I-지속력', 'B-머릿결관리', 'I-머릿결관리', 'B-염색력', 'I-염색력', 'B-발색력', 'I-발색력', 
#                          'B-보습력/수분감/쿨링감', 'I-보습력/수분감/쿨링감', 'B-절삭력', 'I-절삭력', 'B-신축성', 'I-신축성', 'B-소비전력', 'I-소비전력', 
#                          'B-탈취/제습력', 'I-탈취/제습력', 'B-커버력', 'I-커버력', 'B-밀착력/접착력', 'I-밀착력/접착력', 'B-농축도/수용성', 'I-농축도/수용성', 
#                          'B-탈모개선', 'I-탈모개선', 'B-스타일링효과', 'I-스타일링효과', 'B-기능성', 'I-기능성', 'B-세팅력/고정력', 'I-세팅력/고정력', 
#                          'B-청량감/쿨링감', 'I-청량감/쿨링감', 'B-세정력', 'I-세정력', 'B-탄력', 'I-탄력', 'B-코팅력', 'I-코팅력', 'B-거품력', 'I-거품력', 
#                          'B-씨제거력', 'I-씨제거력', 'B-수분감/보습력', 'I-수분감/보습력', 'B-밀착감/접착력', 'I-밀착감/접착력', 'B-분사력', 'I-분사력', 
#                          'B- 클렌징/제거력', 'I-클렌징/제거력', 'B-열전도성', 'I-열전도성', 'B-흡착/접착력', 'I-흡착/접착력', 'B-피부(손)보호', 'I-피부(손)보호', 
#                          'B-두피보호', 'I-두피보호', 'B-살균/소독', 'I-살균/소독', 'B-제연/냄새방지', 'I-제연/냄새방지', 'B-방수성', 'I-방수성', 
#                          'B-위생/살균/성분', 'I-위생/살균/성분', 'B-흡·접착력', 'I-흡·접착력', 'B-향/냄새', 'I-향/냄새', 'B-소음', 'I-소음', 'B-향', 'I-향', 
#                          'B-착용감', 'I-착용감', 'B-자극성', 'I-자극성', 'B-착화감', 'I-착화감', 'B-핏', 'I-핏', 'B-요리력', 'I-요리력', 'B-촉감/감촉', 'I-촉감/감촉', 
#                          'B-촉감', 'I-촉감', 'B-사용감', 'I-사용감', 'B-조립성', 'I-조립성', 'B-호흡성/통기성', 'I-호흡성/통기성', 'B-냄새', 'I-냄새', 'B-이염', 'I-이염', 
#                          'B-그립감', 'I-그립감', 'B-피부타입', 'I-피부타입', 'B-편의성', 'I-편의성', 'B-편의성/사용성/활용도', 'I-편의성/사용성/활용도', 
#                          'B-활용성', 'I-활용성', 'B-수납/건조공간', 'I-수납/건조공간', 'B-사용성', 'I-사용성', 'B-사용성/편의성', 'I-사용성/편의성', 
#                          'B-연마용이성', 'I-연마용이성', 'B-세척용이성', 'I-세척용이성', 'B-정리성/수납력', 'I-정리성/수납력', 'B-제조일/제조사', 'I-제조일/제조사', 
#                          'B-서비스', 'I-서비스', 'B-유통기한', 'I-유통기한']
# print(len(original_sentiment_labels))
# print(len(original_aspect2_list))
# print(len(original_aspect1_list))

# 12. 예측 결과 매핑 및 엑셀 기록
def extract_between_zeros(input_list):
    try:
        first_zero_idx = input_list.index(0)
        second_zero_idx = input_list.index(0, first_zero_idx + 1)
        return input_list[first_zero_idx+1:second_zero_idx]
    except ValueError:
        return []

for i, (sent, asp, asp2) in enumerate(zip(all_sentiments, all_aspects, all_aspects2)):
    sent = extract_between_zeros(sent)
    asp = extract_between_zeros(asp)  # Aspect1 예측 결과
    asp2 = extract_between_zeros(asp2)  # Aspect2 예측 결과

    # 토크나이즈된 텍스트 정리
    tokenized_text = tokenizer.convert_ids_to_tokens(inputs['input_ids'][i].tolist(), skip_special_tokens=True)
    
    cleaned_tokenized_text = [tok for idx, tok in enumerate(tokenized_text) if sent[idx] != 0]

    # 엑셀에 결과 기록
    ws.append(["Sentiment Origin Labels"] + sent)
    ws.append(["Sentiment Mapped Labels"] + [original_sentiment_labels[idx] for idx in sent if idx != 0])
    ws.append(["Aspect2 Original Lables"] + asp2)
    ws.append(["Aspect2 Mapped Labels"] + [original_aspect2_list[idx] for idx in asp2 if idx != 0 ])
    ws.append(["Aspect1 Original Labels"] + asp)
    ws.append(["Aspect1 Mapped Labels"] + [a[idx] for idx in asp if idx != 0])
    ws.append(["Tokenized Text"] + cleaned_tokenized_text)
    ws.append([])  # 빈 줄로 구분


# 13. 엑셀 파일 저장
output_file_excel = rf"\Users\kyn03\Downloads\분석\{original_file_name}.xlsx"
wb.save(output_file_excel)
print(f"[INFO] 엑셀 파일 '{output_file_excel}'로 저장 완료.")



C:\Users\kyn03\AppData\Local\Temp\ipykernel_9540\3008173376.py:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_weights, map_location

['PAD', 'O', 'B-가격', 'I-가격', 'B-품질', 'I-품질', 'B-디자인', 'I-디자인', 'B-색상', 'I-색상', 'B-제품구성', 'I-제품구성', 'B-용기', 'I-용기', 'B-마감', 'I-마감', 'B-포장', 'I-포장', 'B-굽', 'I-굽', 'B-수납', 'I-수납', 'B-품질/디자인/구성', 'I-품질/디자인/구성', 'B-소재', 'I-소재', 'B-제형', 'I-제형', 'B-성분', 'I-성분', 'B-성분/재질', 'I-성분/재질', 'B-재질/소재', 'I-재질/소재', 'B-재질/질감', 'I-재질/질감', 'B-사이즈', 'I-사이즈', 'B-무게', 'I-무게', 'B-용량', 'I-용량', 'B-용량/사이즈', 'I-용량/사이즈', 'B-두께', 'I-두께', 'B-치수/사이즈', 'I-치수/사이즈', 'B-길이', 'I-길이', 'B-사이즈/부피', 'I-사이즈/부피', 'B-사이즈/용량', 'I-사이즈/용량', 'B-사이즈/폭/길이/두께', 'I-사이즈/폭/길이/두께', 'B-용량/수량', 'I-용량/수량', 'B-용량/개수', 'I-용량/개수', 'B-사이즈/두께', 'I-사이즈/두께', 'B-길이/폭/두께', 'I-길이/폭/두께', 'B-깊이/높이', 'I-깊이/높이', 'B-수량/개수', 'I-수량/개수', 'B-기능', 'I-기능', 'B-조작성', 'I-조작성', 'B-기능/효과', 'I-기능/효과', 'B-음량/음질', 'I-음량/음질', 'B-화질', 'I-화질', 'B-효과/성능/기능', 'I-효과/성능/기능', 'B-세척/세정력', 'I-세척/세정력', 'B-시간/속도', 'I-시간/속도', 'B-보습력/수분감', 'I-보습력/수분감', 'B-흡수력', 'I-흡수력', 'B-내구성', 'I-내구성', 'B-지속력/유지력', 'I-지속력/유지력', 'B-발림성', 'I-발림성', 'B-배터리', 'I-배터리', 'B-세정력/청결감', 'I-세정력/청결감', 'B-윤기/피부(톤)

c:\Users\kyn03\anaconda3\Lib\site-packages\transformers\models\bert\modeling_bert.py:440: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
c:\Users\kyn03\anaconda3\Lib\site-packages\torchcrf\__init__.py:317: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorCompare.cpp:530.)
  score = torch.where(mask[i].unsqueeze(1), next_score, score)


KeyboardInterrupt: 

# 바이오힐보

In [ ]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertConfig
from torch.utils.data import DataLoader, TensorDataset
import openpyxl
# ABSAModel 가져오기
from modeling.model import ABSAModel

# 1. 모델 파일 경로 설정
model_dir = r"/Users/kyn03/Downloads/NIA_ABSA/AI모델/소스코드/AI모델소스코드/ckpt/result_model/"

# 2. 모델 설정 로드
config = BertConfig.from_pretrained(model_dir + "config.json")
config.init_model_path = 'klue/bert-base'
config.sentiment_in_feature = 768
config.aspect_in_feature = 768
config.sentiment_drop_ratio = 0.1
config.aspect_drop_ratio = 0.1

# 3. ABSAModel 클래스 초기화
model = ABSAModel(config, num_sentiment=8, num_aspect=226, num_aspect2=16)

# 4. 모델 가중치 로드
model_weights = model_dir + "pytorch_model.bin"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load(model_weights, map_location=device), strict=False)
model.to(device)

# 5. Tokenizer 로드
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

# 6. 데이터 로드
input_file = r"\Users\kyn03\OneDrive\바탕 화면\project_file\바이오힐보_최종(1018).xlsx"  # 업로드된 파일 경로 사용
df = pd.read_excel(input_file)
text_data = df['리뷰'].tolist()
# Check for any non-string values
text_data = df['리뷰'].dropna().astype(str).tolist()
text_data = [str(item) for item in df['리뷰'].dropna().tolist()]


# 7. 텍스트 데이터 토크나이징
inputs = tokenizer(text_data, padding=True, truncation=True, return_tensors="pt")
ids, mask = inputs['input_ids'], inputs['attention_mask']

# 8. TensorDataset 및 DataLoader 설정
dataset = TensorDataset(ids, mask)
dataloader = DataLoader(dataset, batch_size=4)

# 9. Aspect Category 매핑 Dictionary 생성
label_map_dict = {
    "가격": ["가격"],
    "품질/디자인/구성": ["품질", "디자인", "색상", "제품구성", "용기", "마감", "포장", "굽", "수납", "품질/디자인/구성",
                  "소재", "제형", "성분", "성분/재질", "재질/소재", "재질/질감"],
    "사이즈/무게/개수": ["사이즈", "무게", "용량", "용량/사이즈", "두께", "치수/사이즈", "길이", "사이즈/부피", "사이즈/용량",
                  "사이즈/폭/길이/두께", "용량/수량", "용량/개수", "사이즈/두께", "길이/폭/두께", "깊이/높이", "수량/개수"],
    "효과/성능/기능": ["기능","조작성","기능/효과","음량/음질","화질","효과/성능/기능","세척/세정력","시간/속도","보습력/수분감",
                 "흡수력","내구성","지속력/유지력","발림성","배터리","세정력/청결감","윤기/피부(톤)","내구성/견고성","지속력",
                 "머릿결관리","염색력","발색력","보습력/수분감/쿨링감","절삭력","신축성","소비전력","탈취/제습력","커버력",
                 "밀착력/접착력","농축도/수용성","탈모개선","스타일링효과","기능성","세팅력/고정력","청량감/쿨링감",
                 "세정력","탄력","코팅력","거품력","씨제거력","수분감/보습력","밀착감/접착력","분사력","클렌징/제거력",
                 "열전도성","흡착/접착력","피부(손)보호","두피보호","살균/소독","제연/냄새방지","방수성","위생/살균/성분","흡·접착력"],
    "사용감/착용감": ["향/냄새", "소음", "향", "착용감", "자극성", "착화감", "핏", "요리력", "촉감/감촉", "촉감", "사용감", "조립성",
                "호흡성/통기성", "냄새", "이염", "그립감", "피부타입"],
    "편의성/활용성": ["편의성", "편의성/사용성/활용도", "편의성/활용성", "활용성", "수납/건조공간", "사용성", "사용성/편의성",
                "연마용이성", "세척용이성", "정리성/수납력"],
    "제조/유통/서비스":["제조일/제조사", "서비스", "유통기한"]}

# 대분류 Asepct Category Dictionary에 BIO tag 적용
label_list,label_changing_rule = [], {}
for key in label_map_dict.keys():
    if key != 'O':
        label_list.extend(['B-' + key, 'I-' + key])
    else:
        label_list.append('O')
for key, labels in label_map_dict.items():
    for label in labels:
        #if key != label:
            for tag in ["B-", "I-"]:
                label_changing_rule[tag + label] = tag + key

a = ["PAD","O"]
aspect1_labels = list(label_changing_rule.keys())
a.extend(aspect1_labels)
print(a)



# 10. 모델 예측 수행
model.eval()
all_sentiments, all_aspects, all_aspects2 = [], [], []

with torch.no_grad():
    for batch in dataloader:
        batch = tuple(t.to(device) for t in batch)
        sentiment, aspect, aspect2 = model(ids=batch[0], mask=batch[1])
        all_sentiments.extend(sentiment)
        all_aspects.extend(aspect)
        all_aspects2.extend(aspect2)

# 11. 엑셀 파일 생성
wb = openpyxl.Workbook()
ws = wb.active
ws.title = "Sentiment_Aspect_Results"

original_sentiment_labels = ["PAD","O","B-긍정", "I-긍정", "B-부정", "I-부정", "B-중립", "I-중립"]
original_aspect2_list = ['PAD','O','B-가격', 'I-가격', 'B-품질/디자인/구성', 'I-품질/디자인/구성', 'B-사이즈/무게/개수', 'I-사이즈/무게/개수', 
                         'B-효과/성능/기능', 'I-효과/성능/기능', 
                         'B-사용감/착용감', 'I-사용감/착용감', 'B-편의성/활용성', 'I-편의성/활용성', 'B-제조/유통/서비스', 'I-제조/유통/서비스']
# original_aspect1_list = ['PAD', 'O', 
#                          'B-가격', 'I-가격',
#                          'B-품질', 'I-품질', 'B-디자인', 'I-디자인', 'B-색상', 'I-색상', 'B-제품구성', 'I-제품구성', 'B-용기', 'I-용기', 
#                          'B-마감', 'I-마감', 'B-포장', 'I-포장', 'B-굽', 'I-굽', 'B-수납', 'I-수납', 'B-소재', 'I-소재', 'B-제형', 'I-제형', 'B-성분', 'I-성분', 
#                          'B-성분/재질', 'I-성분/재질', 'B-재질/소재', 'I-재질/소재', 'B-재질/질감', 'I-재질/질감', 'B-사이즈', 'I-사이즈', 'B-무게', 'I-무게', 
#                          'B-용량', 'I-용량', 'B-용 량/사이즈', 'I-용량/사이즈', 'B-두께', 'I-두께', 'B-치수/사이즈', 'I-치수/사이즈', 'B-길이', 'I-길이', 
#                          'B-사이즈/부피', 'I-사이즈/부피', 'B-사이즈/용량', 'I-사이즈/용량', 'B-사이즈/폭/길이/두께', 'I-사이즈/폭/길이/두께', 
#                          'B-용량/수량', 'I-용량/수량', 'B-용량/개수', 'I-용량/개수', 'B-사이즈/두께', 'I-사이즈/두께', 'B-길이/폭/두께', 'I-길이/폭/두께', 
#                          'B-깊이/높이', 'I-깊이/높이', 'B-수량/개수', 'I-수량/개수', 'B-기능', 'I-기능', 'B-조작성', 'I-조작성', 'B-기능/효과', 'I-기능/효과', 
#                          'B-음량/음질', 'I-음량/음질', 'B-화질', 'I-화질', 'B-세척/세정력', 'I-세척/세정력', 'B-시간/속도', 'I-시간/속도', 
#                          'B-보습력/수분감', 'I-보습력/수분감', 'B-흡수력', 'I-흡수력', 'B-내구성', 'I-내구성', 'B-지속력/유지력', 'I-지속력/유지력', 
#                          'B-발림성', 'I-발림성', 'B-배터리', 'I-배터리', 'B-세정력/청결감', 'I-세정력/청결감', 'B-윤기/피 부(톤)', 'I-윤기/피부(톤)', 
#                          'B-내구성/견고성', 'I-내구성/견고성', 'B-지속력', 'I-지속력', 'B-머릿결관리', 'I-머릿결관리', 'B-염색력', 'I-염색력', 'B-발색력', 'I-발색력', 
#                          'B-보습력/수분감/쿨링감', 'I-보습력/수분감/쿨링감', 'B-절삭력', 'I-절삭력', 'B-신축성', 'I-신축성', 'B-소비전력', 'I-소비전력', 
#                          'B-탈취/제습력', 'I-탈취/제습력', 'B-커버력', 'I-커버력', 'B-밀착력/접착력', 'I-밀착력/접착력', 'B-농축도/수용성', 'I-농축도/수용성', 
#                          'B-탈모개선', 'I-탈모개선', 'B-스타일링효과', 'I-스타일링효과', 'B-기능성', 'I-기능성', 'B-세팅력/고정력', 'I-세팅력/고정력', 
#                          'B-청량감/쿨링감', 'I-청량감/쿨링감', 'B-세정력', 'I-세정력', 'B-탄력', 'I-탄력', 'B-코팅력', 'I-코팅력', 'B-거품력', 'I-거품력', 
#                          'B-씨제거력', 'I-씨제거력', 'B-수분감/보습력', 'I-수분감/보습력', 'B-밀착감/접착력', 'I-밀착감/접착력', 'B-분사력', 'I-분사력', 
#                          'B- 클렌징/제거력', 'I-클렌징/제거력', 'B-열전도성', 'I-열전도성', 'B-흡착/접착력', 'I-흡착/접착력', 'B-피부(손)보호', 'I-피부(손)보호', 
#                          'B-두피보호', 'I-두피보호', 'B-살균/소독', 'I-살균/소독', 'B-제연/냄새방지', 'I-제연/냄새방지', 'B-방수성', 'I-방수성', 
#                          'B-위생/살균/성분', 'I-위생/살균/성분', 'B-흡·접착력', 'I-흡·접착력', 'B-향/냄새', 'I-향/냄새', 'B-소음', 'I-소음', 'B-향', 'I-향', 
#                          'B-착용감', 'I-착용감', 'B-자극성', 'I-자극성', 'B-착화감', 'I-착화감', 'B-핏', 'I-핏', 'B-요리력', 'I-요리력', 'B-촉감/감촉', 'I-촉감/감촉', 
#                          'B-촉감', 'I-촉감', 'B-사용감', 'I-사용감', 'B-조립성', 'I-조립성', 'B-호흡성/통기성', 'I-호흡성/통기성', 'B-냄새', 'I-냄새', 'B-이염', 'I-이염', 
#                          'B-그립감', 'I-그립감', 'B-피부타입', 'I-피부타입', 'B-편의성', 'I-편의성', 'B-편의성/사용성/활용도', 'I-편의성/사용성/활용도', 
#                          'B-활용성', 'I-활용성', 'B-수납/건조공간', 'I-수납/건조공간', 'B-사용성', 'I-사용성', 'B-사용성/편의성', 'I-사용성/편의성', 
#                          'B-연마용이성', 'I-연마용이성', 'B-세척용이성', 'I-세척용이성', 'B-정리성/수납력', 'I-정리성/수납력', 'B-제조일/제조사', 'I-제조일/제조사', 
#                          'B-서비스', 'I-서비스', 'B-유통기한', 'I-유통기한']
# print(len(original_sentiment_labels))
# print(len(original_aspect2_list))
# print(len(original_aspect1_list))

# 12. 예측 결과 매핑 및 엑셀 기록
def extract_between_zeros(input_list):
    try:
        first_zero_idx = input_list.index(0)
        second_zero_idx = input_list.index(0, first_zero_idx + 1)
        return input_list[first_zero_idx+1:second_zero_idx]
    except ValueError:
        return []

for i, (sent, asp, asp2) in enumerate(zip(all_sentiments, all_aspects, all_aspects2)):
    sent = extract_between_zeros(sent)
    asp = extract_between_zeros(asp)  # Aspect1 예측 결과
    asp2 = extract_between_zeros(asp2)  # Aspect2 예측 결과


    

    # # 디버깅용 출력
    # print(f"Aspect1 매핑된 키 값: {mapped_aspect1_labels}")
    

    
    
    # 토크나이즈된 텍스트 정리
    tokenized_text = tokenizer.convert_ids_to_tokens(inputs['input_ids'][i].tolist(), skip_special_tokens=True)
    
    cleaned_tokenized_text = [tok for idx, tok in enumerate(tokenized_text) if sent[idx] != 0]

    # 엑셀에 결과 기록
    ws.append(["Sentiment Origin Labels"] + sent)
    ws.append(["Sentiment Mapped Labels"] + [original_sentiment_labels[idx] for idx in sent if idx != 0])
    ws.append(["Aspect2 Original Lables"] + asp2)
    ws.append(["Aspect2 Mapped Labels"] + [original_aspect2_list[idx] for idx in asp2 if idx != 0 ])
    ws.append(["Aspect1 Original Labels"] + asp)
    ws.append(["Aspect1 Mapped Labels"] + [a[idx] for idx in asp if idx != 0])
    ws.append(["Tokenized Text"] + cleaned_tokenized_text)
    ws.append([])  # 빈 줄로 구분

# 13. 엑셀 파일 저장
output_file = r"\Users\kyn03\OneDrive\바탕 화면\project_file\sorted_boh.xlsx"
wb.save(output_file)

print(f"[INFO] 엑셀 파일 '{output_file}'로 저장 완료.")


C:\Users\kyn03\AppData\Local\Temp\ipykernel_844\2715848986.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_weights, map_location=

['PAD', 'O', 'B-가격', 'I-가격', 'B-품질', 'I-품질', 'B-디자인', 'I-디자인', 'B-색상', 'I-색상', 'B-제품구성', 'I-제품구성', 'B-용기', 'I-용기', 'B-마감', 'I-마감', 'B-포장', 'I-포장', 'B-굽', 'I-굽', 'B-수납', 'I-수납', 'B-품질/디자인/구성', 'I-품질/디자인/구성', 'B-소재', 'I-소재', 'B-제형', 'I-제형', 'B-성분', 'I-성분', 'B-성분/재질', 'I-성분/재질', 'B-재질/소재', 'I-재질/소재', 'B-재질/질감', 'I-재질/질감', 'B-사이즈', 'I-사이즈', 'B-무게', 'I-무게', 'B-용량', 'I-용량', 'B-용량/사이즈', 'I-용량/사이즈', 'B-두께', 'I-두께', 'B-치수/사이즈', 'I-치수/사이즈', 'B-길이', 'I-길이', 'B-사이즈/부피', 'I-사이즈/부피', 'B-사이즈/용량', 'I-사이즈/용량', 'B-사이즈/폭/길이/두께', 'I-사이즈/폭/길이/두께', 'B-용량/수량', 'I-용량/수량', 'B-용량/개수', 'I-용량/개수', 'B-사이즈/두께', 'I-사이즈/두께', 'B-길이/폭/두께', 'I-길이/폭/두께', 'B-깊이/높이', 'I-깊이/높이', 'B-수량/개수', 'I-수량/개수', 'B-기능', 'I-기능', 'B-조작성', 'I-조작성', 'B-기능/효과', 'I-기능/효과', 'B-음량/음질', 'I-음량/음질', 'B-화질', 'I-화질', 'B-효과/성능/기능', 'I-효과/성능/기능', 'B-세척/세정력', 'I-세척/세정력', 'B-시간/속도', 'I-시간/속도', 'B-보습력/수분감', 'I-보습력/수분감', 'B-흡수력', 'I-흡수력', 'B-내구성', 'I-내구성', 'B-지속력/유지력', 'I-지속력/유지력', 'B-발림성', 'I-발림성', 'B-배터리', 'I-배터리', 'B-세정력/청결감', 'I-세정력/청결감', 'B-윤기/피부(톤)

# 아벤느

In [6]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertConfig
from torch.utils.data import DataLoader, TensorDataset
import openpyxl
# ABSAModel 가져오기
from modeling.model import ABSAModel

# 1. 모델 파일 경로 설정
model_dir = r"/Users/kyn03/Downloads/NIA_ABSA/AI모델/소스코드/AI모델소스코드/ckpt/result_model/"

# 2. 모델 설정 로드
config = BertConfig.from_pretrained(model_dir + "config.json")
config.init_model_path = 'klue/bert-base'
config.sentiment_in_feature = 768
config.aspect_in_feature = 768
config.sentiment_drop_ratio = 0.1
config.aspect_drop_ratio = 0.1

# 3. ABSAModel 클래스 초기화
model = ABSAModel(config, num_sentiment=8, num_aspect=226, num_aspect2=16)

# 4. 모델 가중치 로드
model_weights = model_dir + "pytorch_model.bin"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.load_state_dict(torch.load(model_weights, map_location=device), strict=False)
model.to(device)

# 5. Tokenizer 로드
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

# 6. 데이터 로드
input_file = r"\Users\kyn03\OneDrive\바탕 화면\project_file\아벤느_최종(1018).xlsx"  # 업로드된 파일 경로 사용
df = pd.read_excel(input_file)
text_data = df['리뷰'].tolist()
# Check for any non-string values
text_data = df['리뷰'].dropna().astype(str).tolist()
text_data = [str(item) for item in df['리뷰'].dropna().tolist()]


# 7. 텍스트 데이터 토크나이징
inputs = tokenizer(text_data, padding=True, truncation=True, return_tensors="pt")
ids, mask = inputs['input_ids'], inputs['attention_mask']

# 8. TensorDataset 및 DataLoader 설정
dataset = TensorDataset(ids, mask)
dataloader = DataLoader(dataset, batch_size=4)

# 9. Aspect Category 매핑 Dictionary 생성
label_map_dict = {
    "가격": ["가격"],
    "품질/디자인/구성": ["품질", "디자인", "색상", "제품구성", "용기", "마감", "포장", "굽", "수납", "품질/디자인/구성",
                  "소재", "제형", "성분", "성분/재질", "재질/소재", "재질/질감"],
    "사이즈/무게/개수": ["사이즈", "무게", "용량", "용량/사이즈", "두께", "치수/사이즈", "길이", "사이즈/부피", "사이즈/용량",
                  "사이즈/폭/길이/두께", "용량/수량", "용량/개수", "사이즈/두께", "길이/폭/두께", "깊이/높이", "수량/개수"],
    "효과/성능/기능": ["기능","조작성","기능/효과","음량/음질","화질","효과/성능/기능","세척/세정력","시간/속도","보습력/수분감",
                 "흡수력","내구성","지속력/유지력","발림성","배터리","세정력/청결감","윤기/피부(톤)","내구성/견고성","지속력",
                 "머릿결관리","염색력","발색력","보습력/수분감/쿨링감","절삭력","신축성","소비전력","탈취/제습력","커버력",
                 "밀착력/접착력","농축도/수용성","탈모개선","스타일링효과","기능성","세팅력/고정력","청량감/쿨링감",
                 "세정력","탄력","코팅력","거품력","씨제거력","수분감/보습력","밀착감/접착력","분사력","클렌징/제거력",
                 "열전도성","흡착/접착력","피부(손)보호","두피보호","살균/소독","제연/냄새방지","방수성","위생/살균/성분","흡·접착력"],
    "사용감/착용감": ["향/냄새", "소음", "향", "착용감", "자극성", "착화감", "핏", "요리력", "촉감/감촉", "촉감", "사용감", "조립성",
                "호흡성/통기성", "냄새", "이염", "그립감", "피부타입"],
    "편의성/활용성": ["편의성", "편의성/사용성/활용도", "편의성/활용성", "활용성", "수납/건조공간", "사용성", "사용성/편의성",
                "연마용이성", "세척용이성", "정리성/수납력"],
    "제조/유통/서비스":["제조일/제조사", "서비스", "유통기한"]}

# 대분류 Asepct Category Dictionary에 BIO tag 적용
label_list,label_changing_rule = [], {}
for key in label_map_dict.keys():
    if key != 'O':
        label_list.extend(['B-' + key, 'I-' + key])
    else:
        label_list.append('O')
for key, labels in label_map_dict.items():
    for label in labels:
        #if key != label:
            for tag in ["B-", "I-"]:
                label_changing_rule[tag + label] = tag + key

a = ["PAD","O"]
aspect1_labels = list(label_changing_rule.keys())
a.extend(aspect1_labels)
print(a)



# 10. 모델 예측 수행
model.eval()
all_sentiments, all_aspects, all_aspects2 = [], [], []

with torch.no_grad():
    for batch in dataloader:
        batch = tuple(t.to(device) for t in batch)
        sentiment, aspect, aspect2 = model(ids=batch[0], mask=batch[1])
        all_sentiments.extend(sentiment)
        all_aspects.extend(aspect)
        all_aspects2.extend(aspect2)

# 11. 엑셀 파일 생성
wb = openpyxl.Workbook()
ws = wb.active
ws.title = "Sentiment_Aspect_Results"

original_sentiment_labels = ["PAD","O","B-긍정", "I-긍정", "B-부정", "I-부정", "B-중립", "I-중립"]
original_aspect2_list = ['PAD','O','B-가격', 'I-가격', 'B-품질/디자인/구성', 'I-품질/디자인/구성', 'B-사이즈/무게/개수', 'I-사이즈/무게/개수', 
                         'B-효과/성능/기능', 'I-효과/성능/기능', 
                         'B-사용감/착용감', 'I-사용감/착용감', 'B-편의성/활용성', 'I-편의성/활용성', 'B-제조/유통/서비스', 'I-제조/유통/서비스']
# original_aspect1_list = ['PAD', 'O', 
#                          'B-가격', 'I-가격',
#                          'B-품질', 'I-품질', 'B-디자인', 'I-디자인', 'B-색상', 'I-색상', 'B-제품구성', 'I-제품구성', 'B-용기', 'I-용기', 
#                          'B-마감', 'I-마감', 'B-포장', 'I-포장', 'B-굽', 'I-굽', 'B-수납', 'I-수납', 'B-소재', 'I-소재', 'B-제형', 'I-제형', 'B-성분', 'I-성분', 
#                          'B-성분/재질', 'I-성분/재질', 'B-재질/소재', 'I-재질/소재', 'B-재질/질감', 'I-재질/질감', 'B-사이즈', 'I-사이즈', 'B-무게', 'I-무게', 
#                          'B-용량', 'I-용량', 'B-용 량/사이즈', 'I-용량/사이즈', 'B-두께', 'I-두께', 'B-치수/사이즈', 'I-치수/사이즈', 'B-길이', 'I-길이', 
#                          'B-사이즈/부피', 'I-사이즈/부피', 'B-사이즈/용량', 'I-사이즈/용량', 'B-사이즈/폭/길이/두께', 'I-사이즈/폭/길이/두께', 
#                          'B-용량/수량', 'I-용량/수량', 'B-용량/개수', 'I-용량/개수', 'B-사이즈/두께', 'I-사이즈/두께', 'B-길이/폭/두께', 'I-길이/폭/두께', 
#                          'B-깊이/높이', 'I-깊이/높이', 'B-수량/개수', 'I-수량/개수', 'B-기능', 'I-기능', 'B-조작성', 'I-조작성', 'B-기능/효과', 'I-기능/효과', 
#                          'B-음량/음질', 'I-음량/음질', 'B-화질', 'I-화질', 'B-세척/세정력', 'I-세척/세정력', 'B-시간/속도', 'I-시간/속도', 
#                          'B-보습력/수분감', 'I-보습력/수분감', 'B-흡수력', 'I-흡수력', 'B-내구성', 'I-내구성', 'B-지속력/유지력', 'I-지속력/유지력', 
#                          'B-발림성', 'I-발림성', 'B-배터리', 'I-배터리', 'B-세정력/청결감', 'I-세정력/청결감', 'B-윤기/피 부(톤)', 'I-윤기/피부(톤)', 
#                          'B-내구성/견고성', 'I-내구성/견고성', 'B-지속력', 'I-지속력', 'B-머릿결관리', 'I-머릿결관리', 'B-염색력', 'I-염색력', 'B-발색력', 'I-발색력', 
#                          'B-보습력/수분감/쿨링감', 'I-보습력/수분감/쿨링감', 'B-절삭력', 'I-절삭력', 'B-신축성', 'I-신축성', 'B-소비전력', 'I-소비전력', 
#                          'B-탈취/제습력', 'I-탈취/제습력', 'B-커버력', 'I-커버력', 'B-밀착력/접착력', 'I-밀착력/접착력', 'B-농축도/수용성', 'I-농축도/수용성', 
#                          'B-탈모개선', 'I-탈모개선', 'B-스타일링효과', 'I-스타일링효과', 'B-기능성', 'I-기능성', 'B-세팅력/고정력', 'I-세팅력/고정력', 
#                          'B-청량감/쿨링감', 'I-청량감/쿨링감', 'B-세정력', 'I-세정력', 'B-탄력', 'I-탄력', 'B-코팅력', 'I-코팅력', 'B-거품력', 'I-거품력', 
#                          'B-씨제거력', 'I-씨제거력', 'B-수분감/보습력', 'I-수분감/보습력', 'B-밀착감/접착력', 'I-밀착감/접착력', 'B-분사력', 'I-분사력', 
#                          'B- 클렌징/제거력', 'I-클렌징/제거력', 'B-열전도성', 'I-열전도성', 'B-흡착/접착력', 'I-흡착/접착력', 'B-피부(손)보호', 'I-피부(손)보호', 
#                          'B-두피보호', 'I-두피보호', 'B-살균/소독', 'I-살균/소독', 'B-제연/냄새방지', 'I-제연/냄새방지', 'B-방수성', 'I-방수성', 
#                          'B-위생/살균/성분', 'I-위생/살균/성분', 'B-흡·접착력', 'I-흡·접착력', 'B-향/냄새', 'I-향/냄새', 'B-소음', 'I-소음', 'B-향', 'I-향', 
#                          'B-착용감', 'I-착용감', 'B-자극성', 'I-자극성', 'B-착화감', 'I-착화감', 'B-핏', 'I-핏', 'B-요리력', 'I-요리력', 'B-촉감/감촉', 'I-촉감/감촉', 
#                          'B-촉감', 'I-촉감', 'B-사용감', 'I-사용감', 'B-조립성', 'I-조립성', 'B-호흡성/통기성', 'I-호흡성/통기성', 'B-냄새', 'I-냄새', 'B-이염', 'I-이염', 
#                          'B-그립감', 'I-그립감', 'B-피부타입', 'I-피부타입', 'B-편의성', 'I-편의성', 'B-편의성/사용성/활용도', 'I-편의성/사용성/활용도', 
#                          'B-활용성', 'I-활용성', 'B-수납/건조공간', 'I-수납/건조공간', 'B-사용성', 'I-사용성', 'B-사용성/편의성', 'I-사용성/편의성', 
#                          'B-연마용이성', 'I-연마용이성', 'B-세척용이성', 'I-세척용이성', 'B-정리성/수납력', 'I-정리성/수납력', 'B-제조일/제조사', 'I-제조일/제조사', 
#                          'B-서비스', 'I-서비스', 'B-유통기한', 'I-유통기한']
# print(len(original_sentiment_labels))
# print(len(original_aspect2_list))
# print(len(original_aspect1_list))

# 12. 예측 결과 매핑 및 엑셀 기록
def extract_between_zeros(input_list):
    try:
        first_zero_idx = input_list.index(0)
        second_zero_idx = input_list.index(0, first_zero_idx + 1)
        return input_list[first_zero_idx+1:second_zero_idx]
    except ValueError:
        return []

for i, (sent, asp, asp2) in enumerate(zip(all_sentiments, all_aspects, all_aspects2)):
    sent = extract_between_zeros(sent)
    asp = extract_between_zeros(asp)  # Aspect1 예측 결과
    asp2 = extract_between_zeros(asp2)  # Aspect2 예측 결과


    

    # # 디버깅용 출력
    # print(f"Aspect1 매핑된 키 값: {mapped_aspect1_labels}")
    

    
    
    # 토크나이즈된 텍스트 정리
    tokenized_text = tokenizer.convert_ids_to_tokens(inputs['input_ids'][i].tolist(), skip_special_tokens=True)
    
    cleaned_tokenized_text = [tok for idx, tok in enumerate(tokenized_text) if sent[idx] != 0]

    # 엑셀에 결과 기록
    ws.append(["Sentiment Origin Labels"] + sent)
    ws.append(["Sentiment Mapped Labels"] + [original_sentiment_labels[idx] for idx in sent if idx != 0])
    ws.append(["Aspect2 Original Lables"] + asp2)
    ws.append(["Aspect2 Mapped Labels"] + [original_aspect2_list[idx] for idx in asp2 if idx != 0 ])
    ws.append(["Aspect1 Original Labels"] + asp)
    ws.append(["Aspect1 Mapped Labels"] + [a[idx] for idx in asp if idx != 0])
    ws.append(["Tokenized Text"] + cleaned_tokenized_text)
    ws.append([])  # 빈 줄로 구분

# 13. 엑셀 파일 저장
output_file = r"\Users\kyn03\OneDrive\바탕 화면\project_file\sorted_avene.xlsx"
wb.save(output_file)

print(f"[INFO] 엑셀 파일 '{output_file}'로 저장 완료.")


C:\Users\kyn03\AppData\Local\Temp\ipykernel_844\1314798137.py:26: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_weights, map_location=

['PAD', 'O', 'B-가격', 'I-가격', 'B-품질', 'I-품질', 'B-디자인', 'I-디자인', 'B-색상', 'I-색상', 'B-제품구성', 'I-제품구성', 'B-용기', 'I-용기', 'B-마감', 'I-마감', 'B-포장', 'I-포장', 'B-굽', 'I-굽', 'B-수납', 'I-수납', 'B-품질/디자인/구성', 'I-품질/디자인/구성', 'B-소재', 'I-소재', 'B-제형', 'I-제형', 'B-성분', 'I-성분', 'B-성분/재질', 'I-성분/재질', 'B-재질/소재', 'I-재질/소재', 'B-재질/질감', 'I-재질/질감', 'B-사이즈', 'I-사이즈', 'B-무게', 'I-무게', 'B-용량', 'I-용량', 'B-용량/사이즈', 'I-용량/사이즈', 'B-두께', 'I-두께', 'B-치수/사이즈', 'I-치수/사이즈', 'B-길이', 'I-길이', 'B-사이즈/부피', 'I-사이즈/부피', 'B-사이즈/용량', 'I-사이즈/용량', 'B-사이즈/폭/길이/두께', 'I-사이즈/폭/길이/두께', 'B-용량/수량', 'I-용량/수량', 'B-용량/개수', 'I-용량/개수', 'B-사이즈/두께', 'I-사이즈/두께', 'B-길이/폭/두께', 'I-길이/폭/두께', 'B-깊이/높이', 'I-깊이/높이', 'B-수량/개수', 'I-수량/개수', 'B-기능', 'I-기능', 'B-조작성', 'I-조작성', 'B-기능/효과', 'I-기능/효과', 'B-음량/음질', 'I-음량/음질', 'B-화질', 'I-화질', 'B-효과/성능/기능', 'I-효과/성능/기능', 'B-세척/세정력', 'I-세척/세정력', 'B-시간/속도', 'I-시간/속도', 'B-보습력/수분감', 'I-보습력/수분감', 'B-흡수력', 'I-흡수력', 'B-내구성', 'I-내구성', 'B-지속력/유지력', 'I-지속력/유지력', 'B-발림성', 'I-발림성', 'B-배터리', 'I-배터리', 'B-세정력/청결감', 'I-세정력/청결감', 'B-윤기/피부(톤)